In [ ]:
from tensorflow.python.summary.summary_iterator import summary_iterator
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Directory containing the TensorBoard files
base_dir = "..."
config = "1010_xsub120_policy"

if not os.path.exists(config):
    os.makedirs(config)
    print(f"Directory '{config}' created successfully.")
else:
    print(f"Directory '{config}' already exists.")


In [ ]:
# load data from tensorboard file
common = ["act", "att_lay", "conv_lay", "drop_prob", "init_lay"]
input_stream = ["blocks_in", "depth_in", "stride_in", "scale_in", "temp_win_in", "graph_dist_in", "reduct_ratio_in"]
main_stream = ["blocks_main", "depth_main", "graph_dist_main", "shrinkage_main", "residual_main", "adaptive_main"]

all_values = []

for root, dirs, files in os.walk(base_dir):
    for dir_name in dirs:
        if dir_name.startswith("_Parameters_"):
            dir_path = os.path.join(root, dir_name)
            files = os.listdir(dir_path)
            # Loop over each file
            for file in files:
                # Load TensorBoard file
                file_path = os.path.join(dir_path, file)
                # print(f"Processing file: {file_path}")

                # Load the TensorBoard logs
                for event in summary_iterator(file_path):
                    last_underscore_index = dir_name.rfind("_")
                    if last_underscore_index != -1:
                        act_value = dir_name[last_underscore_index + 1:]
                    else:
                        act_value = "Unknown"
                    for value in event.summary.value:
                        all_values.append((value.tag, act_value, value.simple_value))

# Sort
sorted_values = sorted(all_values, key=lambda x: (x[0], x[1]))

prev_val = None
index = 0
# Write sorted values to a file
with open("./{}/{}_sorted.txt".format(config, config), "w") as f:
    for tag, act_value, simple_value in sorted_values:
        if act_value != prev_val:
            index = 0
            prev_val = act_value
            num_values = int(sum(1 for tager, _, _ in sorted_values if tager == tag) / 1)

        if index == 0:

            f.write(f"{tag}, {act_value}, x=0, y={1/num_values}\n")
            f.write(f"{tag}, {act_value}, x=1, y={simple_value}\n")
        elif index == 1:
            f.write(f"{tag}, {act_value}, x=2, y={simple_value}\n")
        elif index == 2:
            f.write(f"{tag}, {act_value}, x=3, y={simple_value}\n")

         # Increment index for the next value
        index += 1

In [ ]:
# Plotting


# Read data from the text file
data_file = ".../{}/{}_sorted.txt".format(config, config)
with open(data_file, "r") as f:
    lines = f.readlines()

# Parse the data
grouped_data = {}
for line in lines:
    parts = line.split(", ")
    tag = parts[0]
    act_value = parts[1]
    x, y = int(parts[2].split("=")[1]), float(parts[3].split("=")[1])
    key = (tag, act_value)
    if key not in grouped_data:
        grouped_data[key] = {'x': [], 'y': []}
    grouped_data[key]['x'].append(x)
    grouped_data[key]['y'].append(round(y, 8))

# Define groups
common = ["act", "att_lay", "conv_lay", "drop_prob", "init_lay"]
common_names =  ["Activation layer", "Attention layer", "Conv. layer type", "Dropout probability", "Init layer size"]

input_stream = ["blocks_in", "depth_in", "stride_in", "scale_in", "temp_win_in", "graph_dist_in", "reduct_ratio_in"]
input_stream_names = ["Blocks in", "Depth in", "Stride in", "Scaling factor", "Temporal window", "Graph distance", "Reduction factor"]

main_stream = ["blocks_main", "depth_main", "graph_dist_main", "shrinkage_main", "residual_main", "adaptive_main"]
main_stream_names = ["Blocks main", "Depth main", "Graph distance", "Shrinkage", "Residual layer", "Adaptive"]

optimizer = ["lr", "optimizers", "weight_decay", "momentum", "batch_size"]
optimizer_names = ["Learning rate", "Optimizer", "Weight decay", "Momentum", "Batch size"]

# Initialize dicts
group_data = {
    "Common": [common, common_names],
    "Input Stream": [input_stream, input_stream_names],
    "Main Stream": [main_stream, main_stream_names],
    "Optimizer": [optimizer, optimizer_names]
}


In [ ]:
# Plotting for each group and one pdf
markers = ['o', 's', 'D', '^', 'v', '*', 'x']
font = {'fontname': 'Times New Roman', 'size': 14}
font_bigger = {'fontname': 'Times New Roman', 'size': 16}
for group_name, group_keys in group_data.items():
    plt.figure(figsize=(18, 4))
    plt.suptitle(f'Policy values of the {group_name} group', **font_bigger)
    num_subplots = len(group_keys[0])
    subplot_index = 1
    for i, key in enumerate(group_keys[0], start=1):
        plt.subplot(1, num_subplots, i)
        plt.xlabel('# Controller update', **font)
        # plt.ylabel('Probability')

        if i == 1:
            plt.ylabel('Probability', **font)
        else:
            plt.ylabel('')

        plt.xticks([0, 1, 2, 3])
        plt.title(f'{str(group_keys[1][i-1])}', **font)
        plt.grid(True, linewidth=0.25)

        index_marker = 0
        for i, ((tag, act), values) in enumerate(grouped_data.items()):

            if key in tag:
                # plt.scatter(values['x'], values['y'])
                # Calculate step locations with a margin of 10%
                # step_locations = [values['x'][j] + 0.1 * (values['x'][j + 1] - values['x'][j]) for j in range(len(values['x']) - 1)]

                plt.step(values['x'], values['y'], where="post", label=f'{act}', marker=markers[index_marker])
                index_marker += 1

        plt.yticks(rotation=45)
        plt.legend(loc="upper left", framealpha=0, prop={"family": "Times New Roman", "size": 10})
        subplot_index += 1

    plt.tight_layout()
    plt.savefig("./{}/{}_{}.pdf".format(config, config, group_name), bbox_inches='tight')
    plt.show()

In [ ]:
# merge those

from PyPDF2 import PdfMerger

def merge_pdfs(pdf_paths, output_path):
    merger = PdfMerger()
    for pdf_path in pdf_paths:
        merger.append(pdf_path)
    merger.write(output_path)
    merger.close()

def search_for_pdfs(folder_path):
    pdf_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.pdf'):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

folder_path = "./{}".format(config)
pdf_files = search_for_pdfs(folder_path)
output_path = "./{}/{}_merged_plots.pdf".format(config, config)  # Output path for merged PDF

merge_pdfs(pdf_files, output_path)
print("Done merging...")